In [2]:
# Attribution path: "{image_index}.{layer}.{attribution}.{upsample}.png"
# Image path: "{image_index}.png"
# Attribution metrics: "{image_index}.{layer}.{attribution}.{upsample}.csv"

from utils import _DeepLiftShap, _GradCAMPlusPlus, ERFUpsampling, SimpleUpsampling, ERFUpsamplingFast
from data import PascalVOC2007
from models import vgg11_PascalVOC, vgg_preprocess
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = vgg11_PascalVOC()
model.to(device)
# Load the pretrained weights
model.load_state_dict(torch.load("VGG11_PascalVOC.pt", map_location=device))
model.eval()

print("Model loaded successfully")

Model loaded successfully


In [3]:
test_data = PascalVOC2007("test", transform=vgg_preprocess)
train_data = PascalVOC2007("train", transform=vgg_preprocess)

baseline_dist = baseline_dist = torch.cat([train_data[i][0].unsqueeze(0) for i in range(16)], dim=0).to(device)

Using downloaded and verified file: data\VOCtest_06-Nov-2007.tar
Extracting data\VOCtest_06-Nov-2007.tar to data
Using downloaded and verified file: data\VOCtrainval_06-Nov-2007.tar
Extracting data\VOCtrainval_06-Nov-2007.tar to data


In [15]:
import cv2
def save_image(image: torch.Tensor, path: str) -> None:
    # Assume shape is (C, H, W)
    if len(image.shape) != 3:
        raise ValueError("Image shape should be (C, H, W)")
    
    image = image.permute(1, 2, 0).detach().cpu().numpy()
    image = (image - image.min()) / (image.max() - image.min())
    image = (image * 255).astype("uint8")
    
    # Convert from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    cv2.imwrite(path, image)

In [ ]:
import os
ROOT_PATH = "."
OUTPUT_PATH = os.path.join(ROOT_PATH, "output")

IMAGES_PATH = os.path.join(OUTPUT_PATH, "images")
ATTRIBUTIONS_PATH = os.path.join(OUTPUT_PATH, "attributions")

for dirs in [OUTPUT_PATH, IMAGES_PATH, ATTRIBUTIONS_PATH]:
    if not os.path.exists(dirs):
        os.makedirs(dirs)


for image_index in range(1):

    image, label = test_data[image_index]
    images = image.unsqueeze(0).to(device)
    labels = label.to(device)
    print(image.shape, label.shape)


    image_path = os.path.join(IMAGES_PATH, f"{image_index}.png")
    # Save image to disk
    save_image(image, image_path)

    
    for layer_index in range(len(model.features)-1, len(model.features)):
        layer = model.features[layer_index]
        for attribution in [_DeepLiftShap(), _GradCAMPlusPlus(model, layer)]:
            attribution_name = attribution.__class__.__name__
            for upsample in [SimpleUpsampling((224,224)), ERFUpsamplingFast(model, layer, device)]:
                upsample_name = upsample.__class__.__name__
                attribution_path = os.path.join(ATTRIBUTIONS_PATH, f"{image_index}.{layer_index}.{attribution_name}.{upsample_name}.png")

                attribution_map = attribution.attribute(
                    model=model,
                    input_tensor=images,
                    layer=layer,
                    target=labels,
                    baseline_dist=baseline_dist,
                )

                attribution_map = upsample(attribution_map, images)
                save_image(attribution_map[0], attribution_path)

                metrics_path = os.path.join(ATTRIBUTIONS_PATH, f"{image_index}.{layer_index}.{attribution_name}.{upsample_name}.csv")


torch.Size([3, 224, 224]) torch.Size([1])


c:\Users\Luca\AppData\Local\Programs\Python\Python310\lib\site-packages\captum\attr\_core\deep_lift.py:304: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(


torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])


In [ ]:
def calculate_images(image_index:int, layer_index:int, attribution_name: str, upsample_name: str):
    # Calculate the attribution maps and return it
    image, label = test_data[image_index]
    images = image.unsqueeze(0).to(device)

    layer = model.features[layer_index]

    if attribution_name == "_DeepLiftShap":
        attribution = _DeepLiftShap()
    elif attribution_name == "_GradCAMPlusPlus":
        attribution = _GradCAMPlusPlus(model, layer)
    else:
        raise ValueError(f"Invalid attribution name, got {attribution_name}")
    
    if upsample_name == "SimpleUpsampling":
        upsample = SimpleUpsampling((224,224))
    elif upsample_name == "ERFUpsampling":
        upsample = ERFUpsampling(model, layer, device)
    elif upsample_name == "ERFUpsamplingFast":
        upsample = ERFUpsamplingFast(model, layer, device)
    else:
        raise ValueError(f"Invalid upsample name, got {upsample_name}")
    
    attribution_map = attribution.attribute(
        model=model,
        input_tensor=images,
        layer=layer,
        target=label.to(device),
        baseline_dist=baseline_dist,
    )

    attribution_map = upsample(attribution_map, images)

    return attribution_map
    